## Sentiment Analysis of the SMILE Twitter dataset Using BERT

### BERT (Bidirectional Encoder Representations from Transformers)

Transformer based language representation model

Pretrained using a combination of masked language modeling objective and next sentence prediction on a large corpus comprising the Toronto Book Corpus and Wikipedia.

Aside: *Transformers* are like RNNs but do not need to process input sequentially; instead use an attention mechanism to differentially weight different parts of an input

### Exploratory Data Analysis

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [20]:
#Loading the data

data = pd.read_csv('smile-annotations-final.csv', 
                  names = ['id', 'tweet', 'sentiment'])

data.set_index('id', inplace = True)


In [15]:
#Let's look at the distribution of sentiments in our dataset
data.sentiment.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: sentiment, dtype: int64

In [18]:
#Selecting tweets that have only one sentiment label

sentiments = ['happy', 'not-relevant', 'angry', 'surprise', 'sad', 'disgust']

data = data[data['sentiment'].isin(sentiments)]


In [22]:
#Alternate way to choose single sentiment labels 

data = data[~data.sentiment.str.contains('\|')]
data = data[data.sentiment!='nocode']

In [24]:
data.sentiment.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: sentiment, dtype: int64

Note: We have imbalanced data (many more happy labels when compared to all other labels). We will make the train-val split in a stratified way to account for this imbalance.

In [26]:
#Assign numerical values to sentiment labels

sentiments = data.sentiment.unique()

label_dict = {}

for index, sentiments in enumerate(sentiments):
    label_dict[sentiments] = index

In [27]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [28]:
#Add label column to data

data['label'] = data['sentiment'].map(label_dict)


In [31]:
# Split train-val 

from sklearn.model_selection import train_test_split

Xtr, Xvl, Ytr, Yvl = train_test_split(
    data.index.values,
    data.label.values,
    test_size = .15,
    random_state = 17,
    stratify = data.label.values)

In [33]:
# Add train val labels to data 
data['data_type'] = ['']*data.shape[0]
data.loc[Xtr, 'data_type'] = 'train'
data.loc[Xvl, 'data_type'] = 'val'

In [34]:
data.groupby(['sentiment', 'label','data_type']).count()

tweet
sentiment    label data_type       
angry        2     train         48
                   val            9
disgust      3     train          5
                   val            1
happy        0     train        966
                   val          171
not-relevant 1     train        182
                   val           32
sad          4     train         27
                   val            5
surprise     5     train         30
                   val            5

#### Loading Tokenizer and Encoding our Data

Tokenizer - creates numerical tokens for words

In [36]:
# !pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

     |████████████████████████████████| 3.5 MB 5.8 MB/s                               | 1.2 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 1.9 MB/s            
     |████████████████████████████████| 3.7 MB 649 kB/s            
     |████████████████████████████████| 67 kB 6.2 MB/s             
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
You should consider upgrading via the '/Users/aakritikumar/anaconda3/bin/python -m pip install --upgrade pip' command.


In [37]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', #using all lower case data
    do_lower_case = True
)



#### Convert all our tweet data into encoded form
Tokenizing  - splits strings in sub-word token strings
converts tokens strings to ids and back

In [51]:
def encode_tweets(data):
    return tokenizer.batch_encode_plus(
        data.tweet.values,
        add_special_tokens=True,
        return_attention_mask=True,
        padding = 'max_length',
        max_length = 256,
        return_tensors='pt')

encoded_data_train = encode_tweets(data[data.data_type=='train'])
encoded_data_val = encode_tweets(data[data.data_type=='val'])

In [52]:
input_ids_train = encoded_data_train['input_ids']
attention_mask_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type == 'train'].label.values)


input_ids_val = encoded_data_val['input_ids']
attention_mask_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type == 'val'].label.values)



In [53]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_mask_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_mask_val,
                            labels_val)

#### Setting up BERT Pretrained Model

In [72]:
from transformers import BertForSequenceClassification

Customising the model for our problem - we add a layer (of size 6 because we want to classify the data into 6 different classes) on top of the pretrained Bert model

In [75]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels = len(label_dict),
    output_hidden_states=False # we only want to see the final prediction)
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#### Creating DataLoaders

In [76]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [77]:
batch_size = 8

dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size = batch_size
)

#### Setting Up Optimizer and Scheduler

In [79]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [81]:
optimizer = AdamW( 
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

epochs = 10

#scheduler - adjusts the learning rate as training goes on 
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs #why?
)



#### Performance Metrics

f1 scores vs accuracy for imbalanced data: (need to read)


In [82]:
import numpy as np
from sklearn.metrics import f1_score

In [86]:
#using the label that has maximum probability 
def prep_preds(preds):
    return np.argmax(preds, axis = 1).flatten() 
    

def f1_score_func(preds_flat, labels):
    labels_flat = labels.flatten()
    return f1_score(preds_flat, labels_flat, average = 'weighted') #why weighted?

def accuracy_per_class(preds_flat, labels, label_dict):
    label_dict_inverse = {v: k for k,v in label_dict.items()} #cool way to invert the dict
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        
        print(f'Accuracy of Class {label_dict_inverse[label]} = {len(y_preds[y_preds==label])/len(y_true)}')
        
        

#### Training

In [87]:
def evaluate(dataloader_val):
    #Here we don't need to backpropagate/calculate gradients
    
    model.eval() #sets model in evaluation mode - freezes all weights
    
    loss_train_total = 0 #set inital training loss to zero
    
    for batch in dataloader_val:
        model.zero_grad() #set gradients to zero
        
        inputs = {
            'input_ids'      : batch[0],
            'attention_mask' : batch[1],
            'labels'         : batch[2]
        }
        
        with torch.no_grad():
        
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total +=loss.item()
        label_ids = inputs['labels']
        predictions.append(logits)
        true_vals.append(label_ids)

                
    loss_val_avg = loss_val_total/len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis = 0)
    true_vals = np.concatenate(true_vals, axis = 0)
    
    return loss_val_avg, predictions, true_vals

In [89]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train() #sets model in training mode
    
    loss_train_total = 0 #set inital training loss to zero
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch{:1d}'.format(epoch),
                        leave = False,
#                         disabled = False                       
                       )
    for batch in progress_bar:
        model.zero_grad() #set gradients to zero
        
        inputs = {
            'input_ids'      : batch[0],
            'attention_mask' : batch[1],
            'labels'         : batch[2]
        }
        
        outputs = model(**inputs)

        loss = outputs[0]

        loss_train_total +=loss.item()
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'/Users/aakritikumar/Desktop/Models/BERT_epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    preds_flat = preds_flat(predictions)
    val_f1 = f1_score_func(preds_flat, true_vals)
    tqdm.write(f'Validation Loss: {val.loss}')
    tqdm.write(f'Weighted F1 score: {val_f1}')

KeyboardInterrupt: 